In [1]:
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

# Redes Neurais Artificiais com PyTorch
# Sumário

[1. Operações com Tensores](#heading--1)

[2. DataSets e DataLoaders](#heading--2)

   * [2.1 Carregando um DataSet](#heading--2-1)
    
   * [2.2 Criando um DataSet](#heading--2-2)
    
   * [2.3 Preparando os Dados para Treino com DataLoaders](#heading--2-3)
   
[3. Construindo uma Rede Neural](#heading--3)

   * [3.1 Definindo a Classe](#heading--3-1)

<a id="heading--1"></a>
## Operações com Tensores
Os tensores no PyTorch são matrizes multidimensionais que representam dados. Eles são semelhantes aos arrays do Numpy, mas têm algumas vantagens, como poder ser operados em GPUs e armazenar o histórico de cálculos para facilitar a diferenciação automática.

Os tensores podem ser criados a partir de listas Python, arrays Numpy ou valores aleatórios usando a classe torch.Tensor ou suas subclasses2. Os tensores são a base para o aprendizado profundo com PyTorch, pois permitem a construção e o treinamento de redes neurais complexas.

In [4]:
tensor = torch.rand(3,4)
tensor1 = torch.ones(2,2)

print("Tensor aleatório:\n", tensor)
print("\nTensor preenchido com 1s:\n", tensor1)
print("\n")

#Indexação e fatiamento semelhante a Numpy

print("Indexação e Fatiamento:\n")
tensor = torch.ones(4, 4)
print(f"Primeira Linha: {tensor[0]}")
print(f"Primeira Coluna: {tensor[:, 0]}")
print(f"Última Coluna: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

print("Concatenação:\n")
t1 = torch.cat([tensor, tensor, tensor], dim=0)    #dimensão do novo vetor
print(t1)

Tensor aleatório:
 tensor([[0.6040, 0.9285, 0.9023, 0.4302],
        [0.9853, 0.1960, 0.4399, 0.9621],
        [0.1447, 0.3533, 0.1061, 0.5491]])

Tensor preenchido com 1s:
 tensor([[1., 1.],
        [1., 1.]])


Indexação e Fatiamento:

Primeira Linha: tensor([1., 1., 1., 1.])
Primeira Coluna: tensor([1., 1., 1., 1.])
Última Coluna: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])
Concatenação:

tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


In [5]:
#Mutiplicação Matricial
y0 = tensor.T

y1 = tensor @ tensor.T
y2 = tensor.matmul(tensor.T)
print("\nMatriz Transposta:\n", y0)
print("\nMutiplicação Matricial:\n", y2)


Matriz Transposta:
 tensor([[1., 1., 1., 1.],
        [0., 0., 0., 0.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]])

Mutiplicação Matricial:
 tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])


In [6]:
#Multiplicação termo a termo
z1 = tensor * tensor
z2 = tensor.mul(tensor)
print("Multiplicação termo a termo:\n", z2)

Multiplicação termo a termo:
 tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


<a id="heading--2"></a>
## DataSets e DataLoaders

O código para processar amostras de dados pode ficar confuso e difícil de manter; idealmente, queremos que nosso código de conjunto de dados seja desacoplado do nosso código de treinamento de modelo para melhor legibilidade e modularidade. 

*Dataset* armazena as amostras e seus rótulos correspondentes, e *DataLoader* envolve um iterável em torno do *Dataset* para permitir um fácil acesso às amostras. PyTorch fornece duas primitivas de dados: **torch.utils.data.DataLoader** e __torch.utils.data.Dataset__ que permitem que você use conjuntos de dados pré-carregados, bem como seus próprios dados

<a id="heading--2-1"></a>
### Carregando um DataSet

Aqui está um exemplo de como carregar o conjunto de dados Fashion-MNIST do TorchVision1. Fashion-MNIST é um conjunto de dados de imagens de artigos da Zalando, consistindo de 60.000 exemplos de treinamento e 10.000 exemplos de teste2. Cada exemplo é composto por uma imagem em escala de cinza de 28×28 e um rótulo associado de uma das 10 classes.

Carregamos o conjunto de dados FashionMNIST com os seguintes parâmetros: 

**root** é o caminho onde os dados de treino/teste são armazenados,

**train** especifica o conjunto de dados de treinamento ou teste,

**download=True** baixa os dados da internet se eles não estiverem disponíveis em root.

**transform** e **target_transform** especificam as transformações de características e rótulos

In [7]:
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor


training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

<a id="heading--2-2"></a>
### Criando um Dataset
Para a riação de um objeto que herde a classe Dataset, é necessário a utilização de três funções:

__init__: Fornece o conjunto de dados para a calsse;
__len__: Retorna o tamanho do conjunto de dados;
__getitem__: Retorna o dado de indíce especificado.

Os dados podem ser armazenados em arquivos externos como arquivos CSV, que contém tanto o conjunto de dados como a etiqueta dos dados. Também é possível passar parâmetros que representem funções, para geração de dados na própria criação da classe.


In [8]:
class Dados(Dataset):
    def __init__(media, desvio, qamostras):
        # Cria uma distribuição normal com média 0 e desvio padrão 1
        self.distribuicao = Normal(torch.tensor([0.0]), torch.tensor([1.0]))

        # Gera dez amostras aleatórias
        self.amostras = self.distribuicao.sample((qamostras,))
    
    def __len__(self):
        return len(self.amostras)
    
    def __getitem__(self, i):
        return self.amostras[i]



<a id="heading--2-3"></a>
### Preparando os Dados para Treino com DataLoaders

É interesssante que para o treinamento do modelo, os dados sejam passados em lotes, e a cada _época_, eles sejam reorganizados, para evitar o overfitting - Situação indesejável na qual o modelo se adequa perfeitamente aos dados, mas não consegue generalizar para outros conjuntos de dados, tornando o modelo inútil -, e usar o multiprocessamento do Python para acelerar a recuperação de dados.

DataLoader é um iterável que faz justamente esse processo, em uma API(Interface de Programação de Aplicações) fácil de usar

In [ ]:
from torch.utils.data import DataLoader

#Define um conjunto de dados a partir da classe Dados para treinamento e teste
training_dataset = Dados(0,1, 1000)
test_dataset = Dados(0,1,100)

#Define os dataloaders para treinamento e teste
training_dataloader = DataLoader(training_dataset, batch_size = 200, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 20, shuffle = True)


# Cria um iterador para o DataLoader
data_iter = iter(train_dataloader)

# Obtém o primeiro lote de dados
first_batch = next(data_iter)

# Imprime o formato do primeiro lote de dados
print(first_batch.shape)

<a id="heading--3"></a>
## Construindo uma Rede Neural

As redes neurais são modelos computacionais, formadas por camadas de neurônios artificiais, estruturas que realizam operações paramétricas com os dados de entrada. O namespace __torch.nn__ fornece todos os blocos necessários para a construção de uma rede neural.

Todo módulo no PyTorch é uma subclasse do __nn.Module__, e a rede é basicamente um módulo composto por outros módulos. Aqui, será desenvolvido um modelo de regressão para aproximação de valores aleatórios, com distribuição de probabilidade Gaussiana.

<a id="heading--3-1"></a>
### Definindo a Classe

